# Getting started

Following [this](https://github.com/jakdot/pyactr/wiki/Getting-started-I) tutorial.

## Model 1: introduction to goal buffer and production rules

1. Import actr and initialise the mind.

In [1]:
import pyactr as actr

playing_memory = actr.ACTRModel()

/home/javier/pyactr-demo/lib/python3.6/site-packages/pyactr/simulation.py:10: UserWarning: Simulation cannot start a new window because tkinter is not installed. This does not affect ACT-R models in any way, but you will see no separate window for environment. If you want to change that, install tkinter.
  warnings.warn("Simulation cannot start a new window because tkinter is not installed. This does not affect ACT-R models in any way, but you will see no separate window for environment. If you want to change that, install tkinter.")
/home/javier/pyactr-demo/lib/python3.6/site-packages/pyactr/simulation.py:11: UserWarning: Simulation GUI is set to False.
  warnings.warn("Simulation GUI is set to False.")


2. Give knowledge to the mind.

Set goal first.

In [2]:
actr.chunktype("playgame", "game, activity")
initial_chunk = actr.makechunk(typename="playgame", game="memory")

goal = playing_memory.set_goal("goal")
goal.add(initial_chunk)

We defined the game in the initial chunk, but we didn't set any activity.

In [3]:
print(goal)   # output: {playgame(activity=, game=memory)}

{playgame(activity= , game= memory)}


The following production string says the following (-ish):

    if goal is playgame and game == memory and activity = None:
        playgame.activity = presskey

In [4]:
playing_memory.productionstring(name="startplaying", string="""
    =goal>
    isa playgame
    game memory
    activity None
    ==>
    =goal>
    isa playgame
    activity presskey""")   # to be modified later

{'=goal': playgame(activity= None, game= memory)}
==>
{'=goal': playgame(activity= presskey, game= )}

Now we create a simmulation and run it

In [5]:
simulation_game = playing_memory.simulation()
simulation_game.run()

(0, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0, 'PROCEDURAL', 'RULE SELECTED: startplaying')
(0.05, 'PROCEDURAL', 'RULE FIRED: startplaying')
(0.05, 'goal', 'MODIFIED')
(0.05, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.05, 'PROCEDURAL', 'NO RULE FOUND')


Now we can inspect the final chunk in our goal.

In [6]:
final_chunk = goal.pop()    # we can pop it

In [7]:
# let's check what's inside
print("final_chunk =", final_chunk)

print("Contents in `final_chunk`:")
for slot_value in final_chunk:  # we can iterate over its elements
    print(f'\t{slot_value}')

print("final_chunk.game =", final_chunk.game) # we can access it like a named tuple

final_chunk = playgame(activity= presskey, game= memory)
Contents in `final_chunk`:
	('activity', presskey)
	('game', memory)
final_chunk.game = memory


## Model 2: introduction to the motor module

Up to this point, our mind has no contact with its environment. We will change that by simulating keypresses.

We start by creating our production rule:

In [8]:
playing_memory.productionstring(name="presskey", string="""
=goal>
isa playgame
game memory
activity presskey
==>
+manual>
isa _manual
cmd press_key
key 1""") # this rule is not correct, will fix it later

{'=goal': playgame(activity= presskey, game= memory)}
==>
{'+manual': _manual(cmd= press_key, key= 1)}

Now we add the initial_chunk and run the simulation again.
It should end in an inifinite loop :(

In [9]:
goal.add(initial_chunk)

simulation_game = playing_memory.simulation()
simulation_game.run()

(0, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0, 'PROCEDURAL', 'RULE SELECTED: startplaying')
(0.05, 'PROCEDURAL', 'RULE FIRED: startplaying')
(0.05, 'goal', 'MODIFIED')
(0.05, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.05, 'PROCEDURAL', 'RULE SELECTED: presskey')
(0.1, 'PROCEDURAL', 'RULE FIRED: presskey')
(0.1, 'manual', 'COMMAND: press_key')
(0.1, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.1, 'PROCEDURAL', 'RULE SELECTED: presskey')
(0.15, 'PROCEDURAL', 'RULE FIRED: presskey')
(0.15, 'manual', 'COMMAND: press_key')
(0.15, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.15, 'PROCEDURAL', 'RULE SELECTED: presskey')
(0.2, 'PROCEDURAL', 'RULE FIRED: presskey')
(0.2, 'manual', 'COMMAND: press_key')
(0.2, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.2, 'PROCEDURAL', 'RULE SELECTED: presskey')
(0.25, 'PROCEDURAL', 'RULE FIRED: presskey')
(0.25, 'manual', 'COMMAND: press_key')
(0.25, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.25, 'PROCEDURAL', 'RULE SELECTED: presskey')
(0.3, 'PROCEDURAL', 'RULE FIRED: presskey')
(0

/home/javier/pyactr-demo/lib/python3.6/site-packages/pyactr/simulation.py:169: UserWarning: Process in manual interrupted
  warnings.warn("Process in %s interrupted" % name)


We forgot to remove the activity from the goal module, after it is taken over by the motor module.
We also need to modify the "startplaying" production string, so that it also checks that the motor module is not busy before firing the action.

In [10]:
playing_memory.productionstring(name="presskey", string="""
=goal>
isa  playgame
game memory
activity presskey
==>
+manual>
isa _manual
cmd press_key
key 1
=goal>
isa playgame
activity None""")

playing_memory.productionstring(name="startplaying", string="""
=goal>
isa  playgame
game memory
activity None
?manual>
state free
==>
=goal>
isa playgame
activity presskey""")   # other `states` are "busy" and "error"

{'=goal': playgame(activity= None, game= memory), '?manual': {'state': 'free'}}
==>
{'=goal': playgame(activity= presskey, game= )}

Run simulation again

In [11]:
goal.add(initial_chunk)
simulation_game = playing_memory.simulation()
simulation_game.run()

(0, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0, 'PROCEDURAL', 'RULE SELECTED: startplaying')
(0.05, 'PROCEDURAL', 'RULE FIRED: startplaying')
(0.05, 'goal', 'MODIFIED')
(0.05, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.05, 'PROCEDURAL', 'RULE SELECTED: presskey')
(0.1, 'PROCEDURAL', 'RULE FIRED: presskey')
(0.1, 'goal', 'MODIFIED')
(0.1, 'manual', 'COMMAND: press_key')
(0.1, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.1, 'PROCEDURAL', 'NO RULE FOUND')
(0.35, 'manual', 'PREPARATION COMPLETE')
(0.35, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.35, 'PROCEDURAL', 'NO RULE FOUND')
(0.4, 'manual', 'INITIATION COMPLETE')
(0.4, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.4, 'PROCEDURAL', 'NO RULE FOUND')
(0.5, 'manual', 'KEY PRESSED: 1')
(0.5, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.5, 'PROCEDURAL', 'NO RULE FOUND')
(0.65, 'manual', 'MOVEMENT FINISHED')
(0.65, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.65, 'PROCEDURAL', 'RULE SELECTED: startplaying')
(0.7, 'PROCEDURAL', 'RULE FIRED: startplaying')
(0.7, 'goal', 'M

## Model 3 - introduction to vision

We move on to the [next tutorial](https://github.com/jakdot/pyactr/wiki/Getting-started-II) and start our code from 0.

Our mind can set a goal and move (press a key). However there is no feedback from the environment yet. Let's initialise an environment for it:

In [12]:
import pyactr as actr

environment = actr.Environment()
playing_memory = actr.ACTRModel(environment=environment)

We now specify what the environment will look like. Every dictionary in the list is one set of stimuli that will appear together on the screen (640x360 by default).

In [13]:
# first one is empty on purpose, the second is just a letter A in the center of the screen
memory = [{}, {"A": {'text': 'A', 'position': (100, 100)}}]

We set goals and chunks like in previous example.

- game: the game we are going to play (memory)
- activity: what will the mind be doing (keypress, attend, etc.)
- key: which key is the model going to press
- object: will express the object found by that key press

In [14]:
goal = playing_memory.set_goal('goal')
actr.chunktype('playgame', 'game, activity, key, object')
initial_chunk = actr.makechunk(typename='playgame', game='memory', key='1')
goal.add(initial_chunk)

Now we set the rules. After the keypress, a stimulus should appear on the screen. The mind should pay attention to it:

In [15]:
playing_memory.productionstring(name='startplaying', string='''
=goal>
isa playgame
game memory
activity None
==>
=goal>
isa playgame
activity presskey''') # to be modified later

playing_memory.productionstring(name='presskey', string='''
=goal>
isa playgame
game memory
activity presskey
key =k
==>
+manual>
isa _manual
cmd press_key
key =k
=goal>
isa playgame
activity attend''')

{'=goal': playgame(activity= presskey, game= memory, key= =k, object= )}
==>
{'+manual': _manual(cmd= press_key, key= =k), '=goal': playgame(activity= attend, game= , key= , object= )}

The stimulus will be automatically detected by the `visual_location` buffer. To pay attention to it, next rule will check in the `visual_location` buffer and move attention to the screen position where the object is located.

In [16]:
playing_memory.productionstring(name='attendobject', string='''
=goal>
isa playgame
game memory
activity attend
=visual_location>
isa _visuallocation
?manual>
state free
==>
=goal>
isa playgame
activity storeobject
+visual>
isa _visual
cmd move_attention
screen_pos =visual_location''')

{'=goal': playgame(activity= attend, game= memory, key= , object= ), '=visual_location': _visuallocation(color= , screen_x= , screen_y= , value= ), '?manual': {'state': 'free'}}
==>
{'=goal': playgame(activity= storeobject, game= , key= , object= ), '+visual': _visual(cmd= move_attention, color= , screen_pos= =visual_location, value= )}

After attending to the object, we will store it:

In [17]:
playing_memory.productionstring(name='storeobject', string='''
=goal>
isa playgame
game memory
activity storeobject
=visual>
isa _visual
value =v
==>
=goal>
isa playgame
activity None
object =v''')

{'=goal': playgame(activity= storeobject, game= memory, key= , object= ), '=visual': _visual(cmd= , color= , screen_pos= , value= =v)}
==>
{'=goal': playgame(activity= None, game= , key= , object= =v)}

Now we run the simulation:

In [18]:
simulation = playing_memory.simulation(gui=False, environment_process=environment.environment_process, stimuli=memory, triggers='1')
simulation.run()

(0, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0, 'PROCEDURAL', 'RULE SELECTED: startplaying')
****Environment: {}
(0.05, 'PROCEDURAL', 'RULE FIRED: startplaying')
(0.05, 'goal', 'MODIFIED')
(0.05, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.05, 'PROCEDURAL', 'RULE SELECTED: presskey')
(0.1, 'PROCEDURAL', 'RULE FIRED: presskey')
(0.1, 'goal', 'MODIFIED')
(0.1, 'manual', 'COMMAND: press_key')
(0.1, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.1, 'PROCEDURAL', 'NO RULE FOUND')
(0.35, 'manual', 'PREPARATION COMPLETE')
(0.35, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.35, 'PROCEDURAL', 'NO RULE FOUND')
(0.4, 'manual', 'INITIATION COMPLETE')
(0.4, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.4, 'PROCEDURAL', 'NO RULE FOUND')
(0.5, 'manual', 'KEY PRESSED: 1')
****Environment: {'A': {'text': 'A', 'position': (100, 100)}}
(0.5, 'visual_location', 'ENCODED LOCATION: _visuallocation(color= , screen_x= 100, screen_y= 100, value= )')
(0.5, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.5, 'PROCEDURAL', 'NO RULE FOUND')
(0.65,

In [19]:
goal

{playgame(activity= attend, game= memory, key= 1, object= A)}

We managed to store visual information in our memory. However, this model only supports one piece of information, which is not enough to play our game.

## Model 4: Introduction to declarative memory

Declarative memory lets us to store the chunk from the buffers (goal, visual) before they disappear (i.e., get replaced by new information).

First, we make our `startplaying` rule to fire only at the beginning (no object). Then we define a second rule `continueplaying` when there is a value in the `object` slot.

We leave the other rules intact.

In [20]:
playing_memory.productionstring(name='startplaying', string='''
=goal>
isa playgame
game memory
activity None
object None
==>
=goal>
isa playgame
activity presskey''')

playing_memory.productionstring(name='continueplaying', string='''
=goal>
isa playgame
game memory
activity None
object ~None
==>
=goal>
isa playgame
game memory
activity recall''')


{'=goal': playgame(activity= None, game= memory, key= , object= ~None)}
==>
{'=goal': playgame(activity= recall, game= memory, key= , object= )}

We refer to a new activity `recall`, which will be defined by the following rules:

1. recallvalue: recall a chunk with particular values
2. recallsuccessful: gets fired if something was retrieved. It terminates the simulation.
3. recallfailed: model continues playing if nothing was recalled

In [21]:
playing_memory.productionstring(name='recallvalue', string='''
=goal>
isa playgame
game memory
activity recall
object =val
==>
=goal>
isa playgame
game memory
activity checkrecalled
+retrieval>
isa playgame
object =val''')

playing_memory.productionstring(name='recallsuccessful', string='''
=goal>
isa playgame
game memory
activity checkrecalled
object =val
?retrieval>
buffer full
=retrieval>
isa playgame
key =k
==>
=goal>
isa playgame
key =k
activity done''')

playing_memory.productionstring(name='recallfailed', string='''
=goal>
isa playgame
game memory
activity checkrecalled
key 1
object =val
?retrieval>
state error
==>
~goal>
+goal>
isa playgame
game memory
activity presskey
key 2''') # "~goal>" flushes the goal buffer

{'=goal': playgame(activity= checkrecalled, game= memory, key= 1, object= =val), '?retrieval': {'state': 'error'}}
==>
{'~goal': None, '+goal': playgame(activity= presskey, game= memory, key= 2, object= )}

Create new set of stimuli for our mind model to explore:

In [22]:
memory = [{}, 
    {'A': {'text': 'A', 'position': (100, 100)}}, 
    {'A': {'text': 'A', 'position': (100, 100)}}]

goal.add(initial_chunk)
simulation = playing_memory.simulation(gui=False, 
    environment_process=environment.environment_process,
    stimuli=memory,
    triggers=['1', '2', '3'])

simulation.run(max_time=2)

(0, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0, 'PROCEDURAL', 'RULE SELECTED: startplaying')
****Environment: {}
(0.05, 'PROCEDURAL', 'RULE FIRED: startplaying')
(0.05, 'goal', 'MODIFIED')
(0.05, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.05, 'PROCEDURAL', 'RULE SELECTED: presskey')
(0.1, 'PROCEDURAL', 'RULE FIRED: presskey')
(0.1, 'goal', 'MODIFIED')
(0.1, 'manual', 'COMMAND: press_key')
(0.1, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.1, 'PROCEDURAL', 'NO RULE FOUND')
(0.35, 'manual', 'PREPARATION COMPLETE')
(0.35, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.35, 'PROCEDURAL', 'NO RULE FOUND')
(0.4, 'manual', 'INITIATION COMPLETE')
(0.4, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.4, 'PROCEDURAL', 'NO RULE FOUND')
(0.5, 'manual', 'KEY PRESSED: 1')
****Environment: {'A': {'text': 'A', 'position': (100, 100)}}
(0.5, 'visual_location', 'ENCODED LOCATION: _visuallocation(color= , screen_x= 100, screen_y= 100, value= )')
(0.5, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.5, 'PROCEDURAL', 'NO RULE FOUND')
(0.508

Example of things we can monitor after execution:

1. Our end goal
2. The contents of our declarative memory

In [23]:
print(goal)

{playgame(activity= done, game= memory, key= 1, object= A)}


In [24]:
print(playing_memory.decmem)

{_visual(cmd= , color= , screen_pos= _visuallocation(color= , screen_x= 100, screen_y= 100, value= ), value= A): array([0.7085]), playgame(activity= checkrecalled, game= memory, key= 1, object= A): array([0.9585]), _visual(cmd= move_attention, color= , screen_pos= _visuallocation(color= , screen_x= 100, screen_y= 100, value= ), value= A): array([1.5194])}


### Creating rules in a loop

More suitable to our memory game.

In [25]:
for i in range(1, 9):
    playing_memory.productionstring(name='recallfailed' + str(i), string='''
    =goal>
    isa playgame
    game memory
    activity checkrecalled
    key ''' + str(i) + '''
    object =val
    ?retrieval>
    state error
    ==>
    ~goal>
    +goal>
    isa playgame
    game memory
    activity presskey
    key ''' + str(i+1))

playing_memory.productionstring(name='recallfailed9', string='''
=goal>
isa playgame
game memory
activity checkrecalled
key 0
object =val
?retrieval>
state error
==>
~goal>''')

{'=goal': playgame(activity= checkrecalled, game= memory, key= 0, object= =val), '?retrieval': {'state': 'error'}}
==>
{'~goal': None}

We prepare the simulation:

In [26]:
memory = [{}] + [{i: {'text': i, 'position': (100, 100)}} for i in 'BCDEFGHDIJ']
goal.add(initial_chunk)
playing_memory.retrieval.pop()  # flush retrieval
simulation = playing_memory.simulation(environment_process=environment.environment_process, 
    stimuli=memory,
    triggers=[str(i%10) for i in range(1, 11)] + ['0'])

Instead of running it completely, we go step by step and break it when retrieval is successful:

In [27]:
while True:
    simulation.step()
    print(simulation.current_event)
    if simulation.current_event.action == "RULE FIRED: recallsuccessful":
        break

(0, 'PROCEDURAL', 'CONFLICT RESOLUTION')
Event(time=0, proc='PROCEDURAL', action='CONFLICT RESOLUTION')
Event(time=0, proc='ENVIRONMENT', action='STARTING ENVIRONMENT')
(0, 'PROCEDURAL', 'RULE SELECTED: startplaying')
Event(time=0, proc='PROCEDURAL', action='RULE SELECTED: startplaying')
****Environment: {}
(0.05, 'PROCEDURAL', 'RULE FIRED: startplaying')
Event(time=0.05, proc='PROCEDURAL', action='RULE FIRED: startplaying')
(0.05, 'goal', 'MODIFIED')
Event(time=0.05, proc='goal', action='MODIFIED')
(0.05, 'PROCEDURAL', 'CONFLICT RESOLUTION')
Event(time=0.05, proc='PROCEDURAL', action='CONFLICT RESOLUTION')
(0.05, 'PROCEDURAL', 'RULE SELECTED: presskey')
Event(time=0.05, proc='PROCEDURAL', action='RULE SELECTED: presskey')
(0.1, 'PROCEDURAL', 'RULE FIRED: presskey')
Event(time=0.1, proc='PROCEDURAL', action='RULE FIRED: presskey')
(0.1, 'goal', 'MODIFIED')
Event(time=0.1, proc='goal', action='MODIFIED')
(0.1, 'manual', 'COMMAND: press_key')
Event(time=0.1, proc='manual', action='COMMAN

In [28]:
print(goal)

{playgame(activity= done, game= memory, key= 3, object= D)}


In [29]:
print(simulation.show_time())

6.6279
